In [ ]:
import pickle

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from datetime import datetime, timedelta

# link = 'https://membership.gocrimson.com/program?classificationid=dc42ec33-82df-44ca-b06c-109c3685395d'
# driver.get(link)


In [49]:
chrome_options = Options()
chrome_options.add_experimental_option("debuggerAddress", "localhost:9222")

# service = Service()  # You can specify the path to chromedriver here if needed
# driver = webdriver.Chrome(service=service, options=chrome_options)
driver = webdriver.Chrome(options=chrome_options)

link = 'https://membership.gocrimson.com/program?classificationid=dc42ec33-82df-44ca-b06c-109c3685395d'
driver.get(link)


In [474]:
driver.maximize_window()
cookie_button = driver.find_element(By.ID, 'gdpr-cookie-accept')
cookie_button.click()

In [475]:
elements = driver.find_elements(By.CLASS_NAME, "img-link")
courts = {}
for element in elements:
    court = element.text.split('\n')[0]
    court_link = element.get_attribute('href')
    courts[court] = court_link

In [476]:
    #If select date button
    # select_date_button = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'selectDateBtn')))
    # select_date_button = driver.find_elements(By.CLASS_NAME, 'selectDateBtn')
    # select_date_button.click()
    # next_week = driver.find_elements(By.XPATH,f'//td[@year={year} and @month={month} and @day={day}]')
    # next_week.click()
    # ok_button = driver.find_elements(By.XPATH,'//button[text()="Ok"]')[0].click()

In [477]:
def go_to_right_day(wait):
    given_date = datetime.today() 

    # Just making sure next day is there before we click the show more dates button
    next_day = given_date + timedelta(days=1)
    # Extract next dayyear, month, and day
    nd_year, nd_month, nd_day = next_day.year, next_day.month, next_day.day
    nd_year = str(nd_year)
    nd_month = str(nd_month)
    nd_day = str(nd_day)
    bad_class="single-date-select-mobile"
    xpath = f'//button[@data-year={nd_year} and @data-month={nd_month} and @data-day={nd_day} and not(contains(@class,"{bad_class}"))]'
    next_day = wait.until(EC.presence_of_all_elements_located((By.XPATH, xpath)))
    if len(next_day) != 1:
        print("Next day not working")
    next_day[0].click()

    right_arrow_class = 'single-date-right-arrow'
    xpath = f'//button[contains(@class,"{right_arrow_class}")]'
    right_arrow = driver.find_elements(By.XPATH, xpath)
    if len(right_arrow) != 1:
        print("Right arrow not found, just skip for now")
    else:
        right_arrow[0].click()

    # next_week = driver.find_elements(By.XPATH,)
    # Add 7 days
    # next_week = given_date + timedelta(days=7)
    next_week = given_date + timedelta(days=2)

    
    # Extract year, month, and day
    year, month, day = next_week.year, next_week.month, next_week.day
    year = str(year)
    month = str(month)
    day = str(day)
    bad_class="single-date-select-mobile"
    xpath = f'//button[@data-year={year} and @data-month={month} and @data-day={day} and not(contains(@class,"{bad_class}"))]'
    next_week = wait.until(EC.element_to_be_clickable((By.XPATH, xpath)))
    # if len(next_day) != 1:
    #     print("Next week not working")
    next_week.click()

In [483]:
def wait_for_element(driver, locator, timeout=10):
    try:
        WebDriverWait(driver, timeout).until(
            lambda x: len(x.find_elements(*locator)) > 2
        )
        return True
    except:
        return False

In [ ]:
def make_bookings(court_link):
    bookings_made = []
    wait = WebDriverWait(driver, 10)
    driver.get(court_link)
    go_to_right_day(wait)
    # Find all booking cards
    # time_slots = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "program-instance-card")))
    wait_for_element(driver, (By.CLASS_NAME, "program-instance-card"))
    time_slots = driver.find_elements(By.CLASS_NAME, "program-instance-card")
    
    for time_slot in time_slots:
        # Get the spot availability text
        spots_element = time_slot.find_element(By.CLASS_NAME, "spots-tag")
        spots_text = spots_element.text.strip()
        print(spots_text)
        select_button = time_slot.find_element(By.CLASS_NAME, "program-select-btn")
        # Only proceed if there are spots left
        if "No spots left" not in spots_text and \
                "disabled" not in select_button.get_attribute("class") and \
                select_button.is_enabled():

            # Get the time
            time_element = time_slot.find_element(By.CLASS_NAME, "instance-time-header")
            time_text = time_element.text.strip()

            # Find the correct div with title="Location"
            location_div = time_slot.find_element(By.XPATH, './/div[@title="Location"]')
            # Extract court name from the <p> tag inside
            court_name = location_div.find_element(By.TAG_NAME, "p").text.replace("location_on", "").strip()

            select_button.click()
            # Wait for the "Register" button and click it
            register_button = wait.until(EC.element_to_be_clickable((By.ID, 'registerBtn')))
            register_button.click()
            # Proceed to Checkout
            to_checkout_class = "btn-NextRegistrationStep"
            xpath = f'//button[contains(@class,"{to_checkout_class}")]'
            wait_for_element(driver, (By.XPATH, xpath))
            # to_checkout_button = wait.until(EC.element_to_be_clickable((By.XPATH, xpath)))
            element = driver.find_element(By.XPATH, xpath)
            ActionChains(driver).move_to_element(element).click().perform()

            # Checkout
            checkout_button_id = "checkoutButton"
            checkout_button = wait.until(EC.element_to_be_clickable((By.ID, checkout_button_id)))
            checkout_button.click()

            # Final Proceed to Checkout - Checkout Button LOL
            btn_checkout_cart_id = "btnCheckoutCart"
            btn_checkout_cart = wait.until(EC.element_to_be_clickable((By.ID, btn_checkout_cart_id)))
            btn_checkout_cart.click()

            # Store booking details
            bookings_made.append({"time": time_text, "court_name": court_name})
            print(bookings_made)
            return bookings_made
    return bookings_made



In [485]:
available_times_all = {}
for court, court_link in courts.items():
    court_link = courts['Murr Tennis: Court 6 (1.5 Hours)']
    bookings_made = make_bookings(court_link)



Right arrow not found, just skip for now
No Spots Left
1 Spot Left
[{'time': '1:30 PM - 3:00 PM', 'court_name': 'Indoor Tennis Court 6'}]


StaleElementReferenceException: Message: stale element reference: stale element not found
  (Session info: chrome=133.0.6943.127); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
0   chromedriver                        0x0000000100ef2b58 cxxbridge1$str$ptr + 2724820
1   chromedriver                        0x0000000100eeb1b8 cxxbridge1$str$ptr + 2693684
2   chromedriver                        0x0000000100a51afc cxxbridge1$string$len + 93348
3   chromedriver                        0x0000000100a62054 cxxbridge1$string$len + 160252
4   chromedriver                        0x0000000100a61108 cxxbridge1$string$len + 156336
5   chromedriver                        0x0000000100a58110 cxxbridge1$string$len + 119480
6   chromedriver                        0x0000000100a567b0 cxxbridge1$string$len + 112984
7   chromedriver                        0x0000000100a598e0 cxxbridge1$string$le

In [412]:
cookies = driver.get_cookies()


In [431]:
with open("login_cookies.pkl", "wb") as file:
    pickle.dump([cookies[2]], file)

In [382]:
def test_login_cookie(driver, cookie):
    driver.delete_all_cookies()  # Clear all cookies
    driver.add_cookie(cookie)    # Add a single cookie
    driver.refresh()             # Refresh the page




In [391]:
cookie = cookies[2]
test_login_cookie(driver, cookie)



In [398]:
cookies = pickle.load(open("login_cookie.pkl", "rb"))


In [425]:
cookies = driver.get_cookies().copy()

In [426]:
cookies.pop(5)

{'domain': 'membership.gocrimson.com',
 'httpOnly': True,
 'name': '__RequestVerificationToken',
 'path': '/',
 'sameSite': 'Lax',
 'secure': True,
 'value': 'Bu2Znfni7ltG7yYnxdAJmoaJiHAX4fPoCRLhcvTTiMUlUhUEiSzVju2DEPWVdREJTonOKloUc7IUd0Kt6VYx2idmK_tMTjtDdcFMrSzYiDM1'}

In [ ]:
cookies.remove(
    {'domain': '.membership.gocrimson.com',
  'httpOnly': True,
  'name': '.AspNet.ApplicationCookie',
  'path': '/',
  'sameSite': 'None',
  'secure': True,
  'value': 'sU7_C9SBQuOhqUWXGI1K0_ToMOy8iKybo2HuQrtyqw3EPYwdHdXTCKiGyqVAF7smVxTefuE5xnh4eT_Vomgk-qv-TSdxHQnKIAnnryHIBwBgNBHLTbxuh7teSzCoq6PiD88NZUrP5XmLKRxpUo5DhACj2dl4vPaXbvy5-AJOXvcqSXTu5wi_vsLO11EUIgibIl_MYrfAJM3dLq2fE-rYJiyY04DzwSiul7zCzNvwKaBbXs_wOitFivzt9xZFoOKjzAK6LJwT6s-sqlBWBO_ouYd6Ifa1ct3dfMA9UWdohFCIb9t1FfNB6exc7jMHYnRlmesS1i45A2UOSoOQbya2n4Fpn4TX-HcZYuWMEpS-6hc'}
)

ValueError: list.remove(x): x not in list

In [429]:
cookies

[{'domain': '.gocrimson.com',
  'expiry': 1774764434,
  'httpOnly': False,
  'name': '_ga',
  'path': '/',
  'sameSite': 'None',
  'secure': True,
  'value': 'GA1.1.1413137966.1740204219'},
 {'domain': '.gocrimson.com',
  'expiry': 1774764434,
  'httpOnly': False,
  'name': '_ga_M56NV9ZJ4H',
  'path': '/',
  'sameSite': 'None',
  'secure': True,
  'value': 'GS1.1.1740204218.1.1.1740204434.0.0.0'},
 {'domain': 'membership.gocrimson.com',
  'httpOnly': True,
  'name': '.AspNet.ApplicationCookie',
  'path': '/',
  'sameSite': 'None',
  'secure': True,
  'value': '5VUrhB1_wOgHQNwoT59IJsrSRPgL2ArjsxMZiEL6DU2knR7HQSUTRsL4qINVRpwmkRLmTOPqAis-dP6pIOc0rv6ipw7GoFjIG-CI4NocTIFzsx8LWOwsdP2-K7jbpNKFsB43lTVuq3DnrI6-7b8yPKVV_4VzMFBH0t9InqSAAjsPpyREqShQiZO93121JI6ZbAvPdoxnwIBDKs4e-Yh0bRMHjDIv8I_ccf1njyJ2fATKvzKHRVx3a8nOOSMBUAINdZOPb9RStUOxbjLI1ucZJsPlJTjzNJ1wk_VwELb0EOpBIudwR-WrwnmdwV6beh0Tc4lIzeuHQD55Fk83wvhXppsxpTQHG8tK2qfSQ2D0hME'},
 {'domain': '.gocrimson.com',
  'expiry': 1740290834,
  'httpOnly'

In [434]:
driver.get_cookies()

[{'domain': '.gocrimson.com',
  'expiry': 1774764819,
  'httpOnly': False,
  'name': '_ga',
  'path': '/',
  'sameSite': 'None',
  'secure': True,
  'value': 'GA1.1.1413137966.1740204219'},
 {'domain': '.gocrimson.com',
  'expiry': 1774764819,
  'httpOnly': False,
  'name': '_ga_M56NV9ZJ4H',
  'path': '/',
  'sameSite': 'None',
  'secure': True,
  'value': 'GS1.1.1740204218.1.1.1740204819.0.0.0'},
 {'domain': '.gocrimson.com',
  'expiry': 1740291219,
  'httpOnly': False,
  'name': '_gid',
  'path': '/',
  'sameSite': 'None',
  'secure': True,
  'value': 'GA1.2.1325973183.1740204219'},
 {'domain': '.membership.gocrimson.com',
  'httpOnly': True,
  'name': '.AspNet.ApplicationCookie',
  'path': '/',
  'sameSite': 'None',
  'secure': True,
  'value': '5VUrhB1_wOgHQNwoT59IJsrSRPgL2ArjsxMZiEL6DU2knR7HQSUTRsL4qINVRpwmkRLmTOPqAis-dP6pIOc0rv6ipw7GoFjIG-CI4NocTIFzsx8LWOwsdP2-K7jbpNKFsB43lTVuq3DnrI6-7b8yPKVV_4VzMFBH0t9InqSAAjsPpyREqShQiZO93121JI6ZbAvPdoxnwIBDKs4e-Yh0bRMHjDIv8I_ccf1njyJ2fATKvzKHR

In [452]:
from selenium import webdriver

driver = webdriver.Chrome()
browser_version = driver.capabilities['browserVersion']
print(f"Chrome browser version: {browser_version}")


Chrome browser version: 133.0.6943.127


In [453]:
import undetected_chromedriver as uc
print(uc.__version__)

3.5.5


In [13]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

chrome_options = Options()
chrome_options.add_experimental_option("debuggerAddress", "localhost:9222")

driver = webdriver.Chrome(options=chrome_options)
driver.get("google.com")


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=133.0.6943.127)
Stacktrace:
0   chromedriver                        0x000000010341ab58 cxxbridge1$str$ptr + 2724820
1   chromedriver                        0x00000001034131b8 cxxbridge1$str$ptr + 2693684
2   chromedriver                        0x0000000102f79afc cxxbridge1$string$len + 93348
3   chromedriver                        0x0000000102f534d4 chromedriver + 128212
4   chromedriver                        0x0000000102fe8b24 cxxbridge1$string$len + 548044
5   chromedriver                        0x0000000103001694 cxxbridge1$string$len + 649276
6   chromedriver                        0x0000000102fb4ba0 cxxbridge1$string$len + 335176
7   chromedriver                        0x00000001033e3610 cxxbridge1$str$ptr + 2498188
8   chromedriver                        0x00000001033e6924 cxxbridge1$str$ptr + 2511264
9   chromedriver                        

In [17]:
from selenium.common.exceptions import NoSuchWindowException
try:
    driver = webdriver.Chrome(options=chrome_options)
    driver.get("https://www.google.com")  # Use the full URL
    
    # Wait for a specific element to be present before proceeding
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.NAME, "q"))
    )
    print("Page loaded successfully")
except NoSuchWindowException:
    print("NoSuchWindowException occurred. Make sure Chrome is running in debug mode.")
except Exception as e:
    print(f"An error occurred: {str(e)}")
finally:
    if 'driver' in locals():
        driver.quit()

Page loaded successfully
